In [1]:
#from  https://www.slideshare.net/databricks/data-wrangling-with-pyspark-for-data-scientists-who-know-pandas-with-andrew-ray
path = '/home/yang/derby.log'

In [2]:
text_file = sc.textFile(path)
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

In [7]:
print counts.collect()

[(u'----------------------------------------------------------------', 1), (u'', 2), (u'user.dir=/home/yang', 1), (u'15', 1), (u'os.version=4.4.0-83-generic', 1), (u'from', 1), (u'19:46:13', 1), (u'with', 1), (u'-', 4), (u'Jul', 1), (u'started', 1), (u"derby.database.classpath=''", 1), (u'directory', 1), (u'org.apache.spark.sql.hive.client.IsolatedClientLoader$$anon$1@25510982', 1), (u'Apache', 2), (u'java.runtime.version=1.8.0_111-b14', 1), (u'The', 1), (u'(1704137):', 1), (u'os.arch=amd64', 1), (u'derby.system.home=null', 1), (u'2017:', 1), (u'Corporation', 1), (u'Booting', 1), (u'loader', 1), (u'os.name=Linux', 1), (u'Foundation', 1), (u'file:/opt/spark-2.1.0-bin-hadoop2.7/jars/derby-10.12.1.1.jar', 1), (u'/home/yang/metastore_db', 1), (u'instance', 1), (u'version', 1), (u'Class', 1), (u'10.12.1.1', 1), (u'Database', 1), (u'a816c00e-015d-4612-0884-0000051e0858', 1), (u'Derby', 2), (u'CST', 1), (u'class', 1), (u'Software', 1), (u'on', 1), (u'database', 1), (u'Loader', 1), (u'java.ven

In [8]:
data_train = '/home/yang/toyData/adult_train.csv'
data_test = '/home/yang/toyData/adult_test.csv'

In [18]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
schema = StructType([
    StructField("age",IntegerType()),
    StructField("workclass", StringType()),
    StructField("fnlwgt", DoubleType()),
    StructField("education", StringType()),
    StructField("education-num",IntegerType()),
    StructField("marital-status", StringType()),
    StructField("occupation", StringType()),
    StructField("relationship", StringType()),
    StructField("race", StringType()),
    StructField("sex", StringType()),
    StructField("capital-gain", DoubleType()),
    StructField("capital-loss", DoubleType()),
    StructField("hours-per-week", DoubleType()),
    StructField("native-country", StringType()),
    StructField("label", StringType())
])
     
df = spark.read.csv(data_train, header=True, mode="DROPMALFORMED", schema=schema)

In [19]:
print df.schema

StructType(List(StructField(age,IntegerType,true),StructField(workclass,StringType,true),StructField(fnlwgt,DoubleType,true),StructField(education,StringType,true),StructField(education-num,IntegerType,true),StructField(marital-status,StringType,true),StructField(occupation,StringType,true),StructField(relationship,StringType,true),StructField(race,StringType,true),StructField(sex,StringType,true),StructField(capital-gain,DoubleType,true),StructField(capital-loss,DoubleType,true),StructField(hours-per-week,DoubleType,true),StructField(native-country,StringType,true),StructField(label,StringType,true)))


In [22]:
df.show(1)
df.columns
df.dtypes
df.withColumnRenamed('label','class')

+---+----------+-------+----------+-------------+--------------+-------------+--------------+------+-----+------------+------------+--------------+--------------+------+
|age| workclass| fnlwgt| education|education-num|marital-status|   occupation|  relationship|  race|  sex|capital-gain|capital-loss|hours-per-week|native-country| label|
+---+----------+-------+----------+-------------+--------------+-------------+--------------+------+-----+------------+------------+--------------+--------------+------+
| 39| State-gov|77516.0| Bachelors|           13| Never-married| Adm-clerical| Not-in-family| White| Male|      2174.0|         0.0|          40.0| United-States| <=50K|
+---+----------+-------+----------+-------------+--------------+-------------+--------------+------+-----+------------+------------+--------------+--------------+------+
only showing top 1 row



DataFrame[age: int, workclass: string, fnlwgt: double, education: string, education-num: int, marital-status: string, occupation: string, relationship: string, race: string, sex: string, capital-gain: double, capital-loss: double, hours-per-week: double, native-country: string, class: string]

In [23]:
df.drop('fnlwgt')

DataFrame[age: int, workclass: string, education: string, education-num: int, marital-status: string, occupation: string, relationship: string, race: string, sex: string, capital-gain: double, capital-loss: double, hours-per-week: double, native-country: string, label: string]

In [28]:
poor = df[df.label == '<=50k']
print poor.schema


StructType(List(StructField(age,IntegerType,true),StructField(workclass,StringType,true),StructField(fnlwgt,DoubleType,true),StructField(education,StringType,true),StructField(education-num,IntegerType,true),StructField(marital-status,StringType,true),StructField(occupation,StringType,true),StructField(relationship,StringType,true),StructField(race,StringType,true),StructField(sex,StringType,true),StructField(capital-gain,DoubleType,true),StructField(capital-loss,DoubleType,true),StructField(hours-per-week,DoubleType,true),StructField(native-country,StringType,true),StructField(label,StringType,true)))


In [29]:
poor.fillna(0)

DataFrame[age: int, workclass: string, fnlwgt: double, education: string, education-num: int, marital-status: string, occupation: string, relationship: string, race: string, sex: string, capital-gain: double, capital-loss: double, hours-per-week: double, native-country: string, label: string]

In [34]:
import pyspark.sql.functions as F
df.withColumn('new ',F.log(df.age))

DataFrame[age: int, workclass: string, fnlwgt: double, education: string, education-num: int, marital-status: string, occupation: string, relationship: string, race: string, sex: string, capital-gain: double, capital-loss: double, hours-per-week: double, native-country: string, label: string, new : double]

In [36]:
df.withColumn('haha',F.when(df.age>10,1).otherwise(3))

DataFrame[age: int, workclass: string, fnlwgt: double, education: string, education-num: int, marital-status: string, occupation: string, relationship: string, race: string, sex: string, capital-gain: double, capital-loss: double, hours-per-week: double, native-country: string, label: string, haha: int]

In [ ]:
a.join(b,on = 'test')
a.join(b,a.ee = b.rr)

In [ ]:
#pandas
    df  
    #[Out]#       data1     data2 key1 key2  
    #[Out]# 0  0.439801  1.582861    a  one  
    #[Out]# 1 -1.388267 -0.603653    a  two  
    #[Out]# 2 -0.514400 -0.826736    b  one  
    #[Out]# 3 -1.487224 -0.192404    b  two  
    #[Out]# 4  2.169966  0.074715    a  one  
    df.groupby('key1').apply(mean)  
    #[Out]#          data1     data2  
    #[Out]# key1                      
    #[Out]# a     0.407166  0.351307  
    #[Out]# b    -1.000812 -0.509570  
    df.groupby(['key1','key2']).apply(mean)  
    #[Out]#               data1     data2  
    #[Out]# key1 key2                      
    #[Out]# a    one   1.304883  0.828788  
    #[Out]#      two  -1.388267 -0.603653  
    #[Out]# b    one  -0.514400 -0.826736  
    #[Out]#      two  -1.487224 -0.192404 
    
    pd.pivot_table(df,values ='data1', index='key1', columns='key2',aggfunc = np.sum)  
    #[Out]#          data1               data2            
    #[Out]# key2       one       two       one       two  
    #[Out]# key1                                          
    #[Out]# a     1.304883 -1.388267  0.828788 -0.603653  
    #[Out]# b    -0.514400 -1.487224 -0.826736 -0.192404  
    df.pivot_table(['data1'], index='key1',columns='key2')  
    #[Out]#          data1            
    #[Out]# key2       one       two  
    #[Out]# key1                      
    #[Out]# a     1.304883 -1.388267  
    #[Out]# b    -0.514400 -1.487224  

In [42]:
a = df.groupby(['label','sex']).agg({'age':'mean'})
a.show(10)

+------+-------+------------------+
| label|    sex|          avg(age)|
+------+-------+------------------+
|  >50K|   Male| 44.62578805163614|
|  >50K| Female|42.125530110262936|
| <=50K| Female|36.210800667222685|
| <=50K|   Male|37.147012162876784|
+------+-------+------------------+



In [50]:
#index means groupby, columns means pivot
b = df.groupby(['label']).pivot('sex').mean('age')
b.show(10)

+------+------------------+------------------+
| label|            Female|              Male|
+------+------------------+------------------+
|  >50K|42.125530110262936| 44.62578805163614|
| <=50K|36.210800667222685|37.147012162876784|
+------+------------------+------------------+



In [53]:
b.describe().show()

+-------+------+------------------+------------------+
|summary| label|            Female|              Male|
+-------+------+------------------+------------------+
|  count|     2|                 2|                 2|
|   mean|  null| 39.16816538874281| 40.88640010725646|
| stddev|  null| 4.182345298057493| 5.288293145916191|
|    min| <=50K|36.210800667222685|37.147012162876784|
|    max|  >50K|42.125530110262936| 44.62578805163614|
+-------+------+------------------+------------------+



In [58]:
a.toPandas().hist()

AttributeError: 'numpy.ndarray' object has no attribute 'show'